In [1]:
import json
import pandas as pd
from konlpy.tag import Okt # komoran, han, kkma
from jamo import h2j, j2hcj
import re

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

from tqdm.notebook import tqdm
import numpy as np

# 데이터 가져오기, 학습 데이터셋으로 만들기

## 인벤에서 수집한 데이터 가져오기

In [161]:
immoral = pd.read_csv(r'data\immoral.txt', sep = 'asssssssss', header = None)
immoral.columns = ['text']
immoral['immoral'] = True

C:\Users\user\AppData\Local\Temp\ipykernel_23136\2151298477.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  immoral = pd.read_csv(r'data\immoral.txt', sep = 'asssssssss', header = None)


In [162]:
clean = pd.read_csv(r'data\clean.txt', sep = 'asssssssss', header = None)
clean.columns = ['text']
clean['immoral'] = False

C:\Users\user\AppData\Local\Temp\ipykernel_23136\2827705178.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  clean = pd.read_csv(r'data\clean.txt', sep = 'asssssssss', header = None)


## op.gg 데이터 가져오기

In [164]:
# df_lii.to_csv(r'data/champions.csv', index = False)
df_lii = pd.read_csv(r'data/champions.csv')

## Ai hub 데이터 가져오기

In [154]:
# ai hub train set
li = []
for i in range(1, 6):
    with open(r'.\data\train\labled\talksets-train-' + str(i) + '.json') as f:
        data = json.load(f)
    for d in data:
        for s in d['sentences']:
            li.append([s['origin_text'], s['is_immoral'], s['intensity'], 
                    'CENSURE' in s['types'], 'HATE' in s['types'], 'DISCRIMINATION' in s['types'], 
                    'SEXUAL' in s['types'], 'ABUSE' in s['types'], 'VIOLENCE' in s['types'], 'CRIME' in s['types']])
            
train = pd.DataFrame(li, columns=['text', 'immoral', 'intensity', 'CENSURE', 'HATE', 'DISCRIMINATION', 'SEXUAL', 'ABUSE', 'VIOLENCE', 'CRIME'])
train['text'] = train['text'].apply(lambda x : re.sub('#@.+?#', '', x.replace('#@#', '')))

train['immoral'] = (train['SEXUAL'] == True) | (train['ABUSE'] == True)
train_set = train[['text', 'immoral']]

In [156]:
# ai hub test set
li = []
with open(r'.\data\valid\labled\talksets-train-6.json') as f:
    data = json.load(f)
for d in data:
    for s in d['sentences']:
        li.append([s['origin_text'], s['is_immoral'], s['intensity'], 
                'CENSURE' in s['types'], 'HATE' in s['types'], 'DISCRIMINATION' in s['types'], 
                'SEXUAL' in s['types'], 'ABUSE' in s['types'], 'VIOLENCE' in s['types'], 'CRIME' in s['types']])
test = pd.DataFrame(li, columns=['text', 'immoral', 'intensity', 'CENSURE', 'HATE', 'DISCRIMINATION', 'SEXUAL', 'ABUSE', 'VIOLENCE', 'CRIME'])

test['text'] = test['text'].apply(lambda x : re.sub('#@.+?#', '', x.replace('#@#', '')))

test['immoral'] = (test['SEXUAL'] == True) | (test['ABUSE'] == True)
test_set = test[['text', 'immoral']]

# 데이터 전처리


In [116]:
train = pd.read_csv(r'data/train_final.csv')
test = pd.read_csv(r'data/test_final.csv')

train = train.dropna()
test = test.dropna()
train.shape, test.shape

((369609, 2), (46830, 2))

## 데이터 자모음으로 쪼개기

In [117]:
train['text'] = train['text'].apply(lambda x : j2hcj(h2j(x)))
test['text'] = test['text'].apply(lambda x : j2hcj(h2j(x)))

In [120]:
# 자모음 쪼갠 텍스트를 한글자씩 리스트로 만들어서 데이터셋 만든다
def textToList(text):
    li = []
    for t in text:
        li.append(list(t))
    return li

train_list = textToList(train['text'])
train_label = train['immoral']

test_list = textToList(test['text'])
test_label = test['immoral']

# X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [122]:
tokenizer = Tokenizer(num_words=110)
tokenizer.fit_on_texts(train_list) # 단어 인덱스 구축
text_sequences = tokenizer.texts_to_sequences(train_list) # 문자열 -> 인덱스 리스트
                                                            # '나는 천재다 나는 멋있다' -> [1, 2, 1, 3]
# train data로 fit 시켜준 tokenizer로 test data도 인덱스로 변경
text_sequences_test = tokenizer.texts_to_sequences(test_list)

word_vocab = tokenizer.word_index # 딕셔너리 형태
print("전체 단어 개수: ", len(word_vocab)) # 전체 단어 개수 확인

전체 단어 개수:  166


In [123]:
MAX_SEQUENCE_LENGTH = 100 # 문장 최대 길이

X_train = pad_sequences(text_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='pre') # 문장의 길이가 100 글자가 넘어가면 자르고, 모자르면 0으로 채워 넣는다.
y_train = np.array(train_label) # 각 리뷰의 감정을 넘파이 배열로 만든다.

X_test = pad_sequences(text_sequences_test, maxlen=MAX_SEQUENCE_LENGTH, padding='pre') # 문장의 길이가 100 글자가 넘어가면 자르고, 모자르면 0으로 채워 넣는다.
y_test = np.array(test_label) # 각 리뷰의 감정을 넘파이 배열로 만든다.

print('Shape of train input data tensor:', X_train.shape) # 리뷰 데이터의 형태 확인
print('Shape of train label tensor:', y_train.shape) # 감정 데이터 형태 확인
print('Shape of test input data tensor:', X_test.shape) # 리뷰 데이터의 형태 확인
print('Shape of test label tensor:', y_test.shape) # 감정 데이터 형태 확인

Shape of train input data tensor: (369609, 100)
Shape of train label tensor: (369609,)
Shape of test input data tensor: (46830, 100)
Shape of test label tensor: (46830,)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Conv1D, GlobalMaxPooling1D, Flatten, Dropout
from tensorflow.keras import layers, models

vocab_size = 30000
word_vector_dim = 64

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(len(word_vocab)+1, word_vector_dim, input_shape=(None,)))
model.add(tf.keras.layers.Conv1D(512, 3, activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(3))
model.add(tf.keras.layers.Conv1D(512, 5, activation='relu'))
model.add(tf.keras.layers.MaxPooling1D())
model.add(tf.keras.layers.Conv1D(512, 7, activation='relu'))
model.add(tf.keras.layers.GlobalMaxPooling1D())
model.add(tf.keras.layers.Dropout(rate=0.5))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))


model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error', metrics=['accuracy',
                                                                            tf.keras.metrics.TrueNegatives(name='true_negatives'),
                                                                            tf.keras.metrics.TruePositives(name='true_positives'),
                                                                            tf.keras.metrics.FalseNegatives(name='false_negatives'),
                                                                            tf.keras.metrics.FalsePositives(name='false_positives')])
# 이진 분류이므로 손실함수는 binary_crossentropy 사용, 에폭마다 정확도를 보기 위해 accuracy 적용

In [ ]:
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", 
    factor=0.2, 
    patience=2,
    cooldown=2, 
    min_lr=0
)
model.fit(X_train, y_train, epochs= 100, batch_size = 100, validation_split=0.2, callbacks=[early, reduce_lr])

In [115]:
pred_y_train = model.predict(X_train)
print(model.evaluate(X_train, y_train))

pred_y_test = model.predict(X_test)
print(model.evaluate(X_test, y_test))

  339/13806 [..............................] - ETA: 7:51

KeyboardInterrupt: 

In [16]:
train_org = pd.read_csv(r'data/train_final.csv')
train_org = train_org.dropna()
test_org = pd.read_csv(r'data/train_final.csv')
train_org['pred'] = np.where(pred_y_train > 0.5, 1, 0)
test_org['pred'] = np.where(pred_y_test > 0.5, 1, 0)


In [19]:
from sklearn.metrics import classification_report
print(classification_report(train_org['immoral'], train_org['pred']))
print(classification_report(test_org['immoral'], test_org['pred']))

              precision    recall  f1-score   support

           0       0.75      0.79      0.77    333523
           1       0.24      0.21      0.23    108258

    accuracy                           0.65    441781
   macro avg       0.50      0.50      0.50    441781
weighted avg       0.63      0.65      0.64    441781

              precision    recall  f1-score   support

           0       0.88      0.95      0.91     41924
           1       0.81      0.65      0.72     14718

    accuracy                           0.87     56642
   macro avg       0.84      0.80      0.82     56642
weighted avg       0.86      0.87      0.86     56642

